---
cdt: 2024-09-10T11:02:40
title: Excluding Raw Samples Lacking Metadata
description: "Have found that a number of samples in the 'raw' dataset are lacking metadata. As they have been judged to lack value, they are being excluded for now. This document contains the discussion and code to add them to the excluded list"
project: dataset_EDA
summary: 6 samples were added to the excluded list
---


In [1]:
%reload_ext autoreload
%autoreload 2

import duckdb as db
import polars as pl
from pca_analysis.experiments.constants import db_path

con = db.connect(db_path, read_only=True)



.


In [2]:
null_varietals = con.sql(
"""--sql
SELECT
    *
FROM
    pbl.sample_metadata
WHERE
    varietal IS NULL
AND
    detection = 'raw'
"""
).pl()
null_varietals


detection,acq_date,wine,color,varietal,samplecode,id,sample_num
str,str,str,str,str,str,str,i64
"""raw""","""2023-02-16 02:06:05""",null,null,null,"""18""","""49cb7b13-3646-4e65-83c2-43b492…",13
"""raw""","""2023-02-16 15:35:24""",null,null,null,"""25""","""a7bd7365-981b-49c3-874a-1f7b37…",21
"""raw""","""2023-02-16 17:22:50""",null,null,null,"""27""","""3d094718-2d85-462f-bd56-41622d…",23
"""raw""","""2023-03-15 19:38:07""",null,null,null,"""55""","""6c21c6a3-8b4a-4385-a023-09cbf6…",54
"""raw""","""2023-05-09 03:13:54""",null,null,null,"""112""","""631766f0-2c8e-49fd-9381-5716da…",100
"""raw""","""2023-05-09 04:07:24""",null,null,null,"""113""","""e33cb9ba-fbfc-4c47-906c-44d096…",101


6 samples are missing 'wine', 'color', and varietal values. Why?


In [3]:
con.sql(
"""--sql
SELECT
    *
FROM
    c_sample_tracker
WHERE
    samplecode IN (SELECT samplecode FROM null_varietals)
"""
).pl()


detection,sampler,samplecode,vintage,name,open_date,sampled_date,added_to_cellartracker,notes,size,ct_wine_name
str,str,str,str,str,str,str,str,str,str,str
"""raw""","""jonathan""","""18""","""null""","""empty id, missing wine""",null,null,"""y""",null,"""750""",""" """
"""raw""","""jonathan""","""25""","""nv""","""nv ulysse collin champagne bla…","""2023-02-13""",null,"""y""",null,"""750""",""" """
"""raw""","""jonathan""","""27""","""null""","""mystery""","""2023-02-10""",null,"""y""",null,"""750""",""" """
"""raw""","""jonathan""","""55""","""nv""","""mumm tasmania brut prestige""",null,"""2023-03-13""","""y""",null,"""750""",""" """
"""raw""","""jonathan""","""112""",null,"""clembush""","""2023-05-05""","""2023-05-08""","""n""",null,"""750""",""" """
"""raw""","""jonathan""","""113""",null,"""leflaive macon-verze blanc le …","""2023-04-20""","""2023-05-08""","""n""",null,"""750""",""" """


I assume its because the wines are not in the cellar tracker table. Frankly I dont consider these wines of value due to under-representation of their categories. They can be reintegrated later if necessary.


In [4]:
missing_metadata_excluded_samples = con.sql(
"""--sql
WITH
  commented as (
    SELECT
      sample_num,
      'missing metadata' AS comment,
      'excluding_raw_samples_without_metadata.ipynb' as proof
    FROM
      null_varietals
    )
SELECT
  *
FROM
  commented
"""
).pl()

missing_metadata_excluded_samples


sample_num,comment,proof
i64,str,str
13,"""missing metadata""","""excluding_raw_samples_without_…"
21,"""missing metadata""","""excluding_raw_samples_without_…"
23,"""missing metadata""","""excluding_raw_samples_without_…"
54,"""missing metadata""","""excluding_raw_samples_without_…"
100,"""missing metadata""","""excluding_raw_samples_without_…"
101,"""missing metadata""","""excluding_raw_samples_without_…"


We will thus add them to the excluded list.

In [9]:
#%%script #uncomment to execut this cell

con.close()
con = db.connect(db_path)

if input("warning: this will add values to table 'dataset_eda.excluded_samples'. press 'y' to continue:") == 'y':
    con.sql(
    """--sql
    INSERT INTO
        dataset_eda.excluded_samples
    SELECT
        *
    FROM
        missing_metadata_excluded_samples
    """)

    excluded_samples = con.sql(
    """--sql
    SELECT
        *
    FROM
        dataset_eda.excluded_samples
    """
    ).pl()
    print("added values to 'dataset_eda.excluded_samples'")
    display(excluded_samples)
else:
    print('did not execute SQL')


ConstraintException: Constraint Error: Duplicate key "sample_num: 13" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).